In [1]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
pd.set_option('display.max_columns',107)
pd.set_option("max_rows", 500)
import dask.dataframe as dd
from dask import delayed
import gzip
import glob
import os
import re
import json
import matplotlib.pyplot as plt
%matplotlib inline

## Review Single Datasets

In [2]:
os.chdir('/Users/peace/standalone_projects/airbnb ')

In [ ]:
# reviews=pd.read_csv('data/Barossa Valley/2020-10-26_ reviews.csv')
# reviews_sy=pd.read_csv('data/Sydney/2020-11-05_ reviews.csv')
# reviews_mel=pd.read_csv('data/Melbourne/2020-11-05_ reviews.csv')
# calender=pd.read_csv('data/Sydney/2020-11-05_calendar.csv')
# listings=pd.read_csv('data/Sydney/2020-11-05_listings.csv')
# reviews_big=pd.read_csv('data/Sydney/2020-11-05_reviews.csv')
# listings_big=pd.read_csv('data/Barossa Valley/11-2-2019_listings 2.csv')
# neighbours=pd.read_csv('data/Barossa Valley/neighbourhoods.csv')
# geo_bv=pd.read_json('Data/Barossa Valley/neighbourhoods.geojson')

In [ ]:
# geo_bv.features[0]['geometry']['coordinates']

## Create DataFrames
Aggregate all the data since 2015 (over 1000 files) (some hosts might have taken off their listings during this period, those reviews wouldn't be availabel in the latest csv file)

### Create A Review DataFrame

In [ ]:
# #Use dask dataframe to aggregate over 200 csv files
# def create_reviews():
#     reviews=[]
#     for city in ['Barossa Valley','Barwon South West, Vic','Melbourne','Northern Rivers','Sydney','Tasmania']:
#         path=f'Data/{city}/*_ reviews.csv'
#         reviews_dask=dd.read_csv(path)
#         reviews_dask=reviews_dask.drop_duplicates(ignore_index=True)
#         reviews_dask['city']=city
#         reviews.append(reviews_dask)
#     reviews_final=dd.concat(reviews,axis=0,ignore_index=True)
#     return reviews_final.compute()

In [ ]:
# def create_reviews():
#     reviews_city=[]
#     for city in ['Barossa Valley','Barwon South West, Vic','Melbourne','Northern Rivers','Sydney','Tasmania']:
#         path=glob.glob(f'Data/{city}/*_ reviews.csv')
#         reviews=[]
#         for filename in path:
#             reviews_dask=pd.read_csv(filename)
#             reviews.append(reviews_dask)
#         reviews_ddf=pd.concat(reviews,axis=0,ignore_index=True)
#         reviews_ddf=reviews_ddf.drop_duplicates(ignore_index=True)
#         reviews_ddf['city']=city
#         reviews.append(reviews_ddf)
#     reviews_final=dd.concat(reviews,axis=0,ignore_index=True)
#     return reviews_final.compute()

In [ ]:
# %%time
# reviews_og_df=create_reviews()

In [ ]:
# reviews_og_df.shape

In [ ]:
# reviews_df.shape

In [ ]:
# Save df
# reviews_df.to_csv('Data/reviews.csv',index=False)

### Create A Cancellation Dataframe & A Review Dataframe
A review would be generated automatically when a booking is cancelled by the hosts.

In [3]:
@delayed
def pandas_df_gz(filename):
    df=pd.read_csv(filename,compression='gzip',error_bad_lines=False,engine='python')
#     index_cancel=df[df['comments'].str.contains('This is an automated posting',na=False)].index
    index_review=df[~df['comments'].str.contains('This is an automated posting',na=False)].index
#     cancel_df=df.iloc[index_cancel]
#     cancel_df.drop(['reviewer_id','reviewer_name','comments'],axis=1,inplace=True)
    review_df=df.iloc[index_review]
    review_df.drop(['comments'],axis=1,inplace=True)
    return review_df

In [4]:
# Dask doesn't work on gz files. 
def create_cancels():
#     cancel_city=[]
    review_city=[]
    for city in ['Barossa Valley','Barwon South West, Vic','Melbourne','Northern Rivers','Sydney','Tasmania']:
        path=f'Data/{city}'
        all_files=glob.glob(path+'/*_reviews.csv.gz')
#         cancel=[]
        review=[]
        for filename in all_files:
            split=re.split('/|-|_',filename)
            year=int(split[2])
            if year>=2019:
                review_dask=pandas_df_gz(filename)
#                 cancel_dask=dd.from_delayed(cancel_dask)
#                 cancel.append(cancel_dask)
                review_dask=dd.from_delayed(review_dask)
                review.append(review_dask)
#         cancel_df=dd.concat(cancel,axis=0,ignore_index=True)
#         cancel_df=cancel_df.drop_duplicates(ignore_index=True)
#         cancel_df['city']=city
#         cancel_city.append(cancel_df)
        review_df=dd.concat(review,axis=0,ignore_index=True)
        review_df=review_df.drop_duplicates(ignore_index=True)
        review_df['city']=city
        review_city.append(review_df)
        print(city)
        
#     cancel_final=dd.concat(cancel_city,axis=0,ignore_index=True)
    review_final=dd.concat(review_city,axis=0,ignore_index=True)

    return review_final.compute()

In [5]:
review_df=create_cancels()

/anaconda3/envs/airbnb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Barossa Valley


/anaconda3/envs/airbnb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Barwon South West, Vic


/anaconda3/envs/airbnb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

/anaconda3/envs/airbnb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Sydney


/anaconda3/envs/airbnb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Tasmania


/anaconda3/envs/airbnb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/anaconda3/envs/airbnb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/anaconda3/envs/airbnb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/ana

In [65]:
review_df=review_df.drop(['reviewer_name'],axis=1).drop_duplicates(ignore_index=True)
review_df.drop(['city','reviewer_id','id'],axis=1,inplace=True)

In [70]:
review_df.shape

(2723370, 2)

In [85]:
# Save df
# cancel_df.to_csv('Data/cancels.csv',index=False)
review_df.to_csv('Data/reviews.csv',index=False)
# review_og_df=pd.read_csv('Data/reviews.csv')
# cancel_og_df=pd.read_csv('Data/cancels.csv')

### Create A Listing DataFrame
Join over 20 snapshots for every city from 2019-01 to 2020-11

In [ ]:
@delayed
def pandas_df(filename,date,city):
    df=pd.read_csv(filename)
    df['city']=city
    df['date']=date
    return df

In [ ]:
# ,'Barwon South West, Vic','Melbourne','Northern Rivers','Sydney','Tasmania'
def create_listings():
    listings_city=[]
    for city in ['Barossa Valley','Barwon South West, Vic','Melbourne','Northern Rivers','Sydney','Tasmania']:
        path=glob.glob(f'Data/{city}/*_listings.csv')
        listings=[]
        for filename in path:
            split=re.split('/|-|_',filename)
            year=int(split[2])
            if year>=2019:
                date=re.split('/|_',filename)[2]
                listings_dask=pandas_df(filename,date,city)
                listings_dask=dd.from_delayed(listings_dask)
                listings.append(listings_dask)
        listings_df=dd.concat(listings,axis=0,ignore_index=True)
        listings_city.append(listings_df)
    listings_final=dd.concat(listings_city,axis=0,ignore_index=True)

    return listings_final.compute()

In [ ]:
listings_df=create_listings()

In [ ]:
listings_df['date']=pd.to_datetime(listings_df.date)

In [83]:
listings_df.shape

NameError: name 'listings_df' is not defined

In [ ]:
# Save df
# listings_df.to_csv('Data/listings.csv',index=False)

## Clean Data for Plotly Plots

### reviews_df

In [96]:
reviews_df=pd.read_csv('Data/reviews.csv')  

In [97]:
reviews_df.shape

(2723370, 2)

In [98]:
reviews_df['date_ym']=pd.to_datetime(reviews_df['date']).dt.strftime('%Y-%m')

In [100]:
reviews_df=reviews_df[reviews_df['date_ym']>'2019-06']
# reviews_df.drop(['date'],axis=1,inplace=True)

In [102]:
# Save cleaned df
# reviews_df.to_csv('Data/cleaned_reviews.csv',index=False)

#### Monthly Reviews

In [ ]:
monthly_reviews_df=reviews_df.groupby(['city','date_ym']).count()

#### Monthly Listings' Reviews

In [ ]:
listings_reviews_df=reviews_df.groupby(['listing_id','date_ym']).count()

### cancels_df

In [ ]:
cancels_df=pd.read_csv('Data/cancels.csv')

In [ ]:
cancels_df['date_ym']=pd.to_datetime(cancels_df['date']).dt.strftime('%Y-%m')
cancels_df=cancels_df[cancels_df['date_ym']>'2019']
# cancels_df.drop(['id','date','reviewer_id','reviewer_name','comments'],axis=1,inplace=True)
# cancels_df=cancels_df.groupby(['city','date_ym']).count()

In [ ]:
# Save cleaned df
# cancels_df.to_csv('Data/cleaned_cancels.csv',index=True)

In [ ]:
cancels_df=pd.read_csv('Data/cleaned_cancels.csv')

In [ ]:
cancels_df.shape

### listings_df

In [ ]:
listings_df=pd.read_csv('Data/listings.csv')

In [ ]:
# create a column for only year and month
listings_df.loc[listings_df['date']=='2019-05-01','date']='2019-04-30'
listings_df.loc[listings_df['date']=='2019-09-01','date']='2019-08-31'
listings_df['date_ym']=pd.to_datetime(listings_df['date']).dt.strftime('%Y-%m')
listings_df.drop(['host_id','host_name','neighbourhood_group','number_of_reviews','last_review','reviews_per_month','calculated_host_listings_count','date','neighbourhood','minimum_nights'],axis=1,inplace=True)

In [ ]:
# delete isolated data
index_1=listings_df[(listings_df['city']=='Barossa Valley')&(listings_df['date_ym']=='2020-10')].index.values
index_2=listings_df[(listings_df['city']=='Barwon South West, Vic')&(listings_df['date_ym']=='2020-10')].index.values

In [ ]:
listings_df.drop(index_1,axis=0,inplace=True)
listings_df.drop(index_2,axis=0,inplace=True)

In [ ]:
def update_date(s):
    '''
    create a date number column for the slider widget
    '''
    year=re.split('-',s)[0]
    month=re.split('-',s)[1]
    if year=='2019':
        num=int(month)-7
    elif year=='2020':
        num=int(month)+5
    else:
        print('Error')
    return num

In [ ]:
listings_df['date_num']=listings_df['date_ym'].apply(update_date)

In [ ]:
# Drop rows with 0 availability in 365 days
listings_index=listings_df[listings_df['availability_365']==0].index
listings_df.drop(listings_index,axis=0,inplace=True)
listings_df.drop('availability_365',axis=1,inplace=True)

In [ ]:
listings_df=listings_df[listings_df['date_ym']>'2019-06']

In [ ]:
map_room={'Entire home/apt':'Entire home','Private room':'Private room','Hotel room':'Hotel','Shared room':'Shared'}
listings_df['room_type']=listings_df.room_type.map(map_room)

In [9]:
# Save cleaned df
# listings_df.to_csv('Data/cleaned_listings.csv',index=False)
# listings_df=pd.read_csv('Data/cleaned_listings.csv')